In [1]:
import pandas as pd
import plotly.express as px


In [2]:
# Reading data to dataframe

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

df = pd.read_csv(url)

#df_1 = pd.read_csv("spacex_web_scraped_2.csv")

In [3]:
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
df["Booster Version Category"].unique()

array(['v1.0', 'v1.1', 'FT', 'B4', 'B5'], dtype=object)

In [5]:
# Creating function that makes the two plots for the Dashboard

def saitti_funktio(x):
                 
    if x == 'CCAFS LC-40' or x == 'VAFB SLC-4E' or x == 'KSC LC-39A' or x == 'CCAFS SLC-40':

        frame = df[df["Launch Site"]==x]

        ## PIE ALKAA 

        landing_class_pituus = len(frame["class"])

        onnistuneiden_laskujen_määrä = len(frame[frame["class"]==1])

        epäonnistuneiden_laskujen_määrä = landing_class_pituus - onnistuneiden_laskujen_määrä

        data = {
                "Category": ["Success", "Failure"],
                "Count": [onnistuneiden_laskujen_määrä, epäonnistuneiden_laskujen_määrä]
                }

        df_1 = pd.DataFrame(data)

        fig_pie = px.pie(df_1, names="Category", values="Count", title=f"Launch Results for {x} Site")

        fig_pie.show()

        ## PIE LOPPUU ja toinen kuva alkaa. Saitti on valittuna ja se on "frame"

        fig_scatter = px.scatter(frame, x="Payload Mass (kg)", y="class", color="Booster Version Category")

        fig_scatter.update_layout(
            title=f"Correlation between Payload and Success for Site {x}",
            xaxis_title="Payload Mass (kg)",
            yaxis_title="Class (1 = success, 0 = failure)",
                        )

        fig_scatter.show()

    if x == "All Sites":

        ## PIE ALKAA 

        value_counts_1 = df.groupby("Launch Site")["class"].value_counts()

        result_df = value_counts_1.reset_index(name="Count")

        filtered_df = result_df[result_df['class'] == 1]

        fig_pie = px.pie(filtered_df, names="Launch Site", values="Count", title="Total Success Launches By All Sites")

        fig_pie.show()

        ## PIE LOPPUU ja toinen kuva alkaa.

        fig_scatter = px.scatter(df, x="Payload Mass (kg)", y="class", color="Booster Version Category")

        fig_scatter.update_layout(
            title="Correlation between Payload and Success for All Sites",
            xaxis_title="Payload Mass (kg)",
            yaxis_title="Class (1 = success, 0 = failure)",
                        )

        fig_scatter.show()

    
saitti_funktio("All Sites")

In [6]:
# Importing needed libraries

import pandas as pd
import plotly.express as px
import dash
from dash import html
from dash import dcc
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
import numpy as np

In [7]:
# TEST that had errors and needed correction

min_value = 0  # Set your initial minimum value
max_value = 10000  # Set your initial maximum value

app = dash.Dash(__name__)

app.layout = html.Div(children=[
                                html.H1("SpaceX Launch Records Dashboard",
                                style={'textAlign': 'center', 'color': '#503D36','font-size': 60}),

                                #html.P("Page sub-headline",
                                #style={"textAlign": "center", "color": "green", "font-size": 30}),

                                #html.Br(),

                                html.Div([  
                                    
                                    #html.Div([
                                    #html.H2("Helper text for the dropdowns",
                                    #style={"margin-right": "2em"}),
                                    #]), 

                                    dcc.Dropdown(
                                    id="x-dropdown", # oli sana-dropdown
                                    options=[
                                    {"label": "All Sites", "value": "All Sites"},
                                    {"label": "CCAFS LC-40", "value": "CCAFS LC-40"},
                                    {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"},
                                    {"label": "KSC LC-39A", "value": "KSC LC-39A"},
                                    {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"},
                                    ],
                                    value="All Sites"
                                    ),

                                    dcc.RangeSlider(id='luku-dropdown',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                    value=[min_value, max_value]),
                                        
                                    html.Div([

                                        html.Div([], id="plot1") 
                                            ], style={"display": "flex", "flex-direction": "column"}),

                                        html.Div([], id="plot2") 
                                            ], style={"display": "flex", "flex-direction": "column"}),

                                
                        ])
                
               # ])  # ?? 

@app.callback([
    Output(component_id="plot1", component_property="children"),
    Output(component_id="plot2", component_property="children")],

    [Input(component_id="x-dropdown", component_property="value"),
     Input(component_id="luku-dropdown", component_property="value")],

    [State("plot1","children"),
    State("plot2","children")
          ])

            
def saitti_funktio(x,paino,children1,children2):
        
    if x == 'CCAFS LC-40' or x == 'VAFB SLC-4E' or x == 'KSC LC-39A' or x == 'CCAFS SLC-40':

        frame = df[df["Launch Site"]==x]

        ## PIE ALKAA 

        landing_class_pituus = len(frame["class"])

        onnistuneiden_laskujen_määrä = len(frame[frame["class"]==1])

        epäonnistuneiden_laskujen_määrä = landing_class_pituus - onnistuneiden_laskujen_määrä

        data = {
                "Category": ["Success", "Failure"],
                "Count": [onnistuneiden_laskujen_määrä, epäonnistuneiden_laskujen_määrä]
                }

        df_1 = pd.DataFrame(data)

        fig_pie = px.pie(df_1, names="Category", values="Count", title=f"Launch Results for {x} Site")

        #fig_pie.show()

        ## PIE LOPPUU ja toinen kuva alkaa. Saitti on valittuna ja se on "frame"

        paino_frame = frame[frame["Payload Mass (kg)"]<=paino]
        
        fig_scatter = px.scatter(paino_frame, x="Payload Mass (kg)", y="class", color="Booster Version Category")

        fig_scatter.update_layout(
            title=f"Correlation between Payload and Success for Site {x}",
            xaxis_title="Payload Mass (kg)",
            yaxis_title="Class (1 = success, 0 = failure)",
                        )

        #fig_scatter.show()

        return [dcc.Graph(figure=fig_pie),
                dcc.Graph(figure=fig_scatter)]

    if x == "All Sites":

        ## PIE ALKAA 

        value_counts_1 = df.groupby("Launch Site")["class"].value_counts()

        result_df = value_counts_1.reset_index(name="Count")

        filtered_df = result_df[result_df['class'] == 1]

        fig_pie = px.pie(filtered_df, names="Launch Site", values="Count", title="Total Success Launches By All Sites")

        #fig_pie.show()

        ## PIE LOPPUU ja toinen kuva alkaa.

        paino_frame = df[df["Payload Mass (kg)"]<=paino]

        fig_scatter = px.scatter(paino_frame, x="Payload Mass (kg)", y="class", color="Booster Version Category")

        fig_scatter.update_layout(
            title="Correlation between Payload and Success for All Sites",
            xaxis_title="Payload Mass (kg)",
            yaxis_title="Class (1 = success, 0 = failure)",
                        )

        #fig_scatter.show()
        
        return [dcc.Graph(figure=fig_pie),
                dcc.Graph(figure=fig_scatter)]


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit


In [34]:
# Full code for the Dashboar that works

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import Input, Output, State
import plotly.express as px

# Initialize your Dash app and DataFrame 
app = dash.Dash(__name__)

# Set initial minimum and maximum values for the RangeSlider
min_value = 0
max_value = 10000

app.layout = html.Div(children=[
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'color': '#503D36', 'font-size': 60}),
    
    dcc.Dropdown(
        id="x-dropdown",
        options=[
            {"label": "All Sites", "value": "All Sites"},
            {"label": "CCAFS LC-40", "value": "CCAFS LC-40"},
            {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"},
            {"label": "KSC LC-39A", "value": "KSC LC-39A"},
            {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"},
        ],
        value="All Sites"
    ),
    
    dcc.RangeSlider(
    id='luku-dropdown',
    min=0, max=10000, step=1000,
    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
    value=[min_value, max_value]
    ),
    
    html.Div([], id="plot1"),
    html.Div([], id="plot2")
])

#@app.callback( # This callback should have same functionality as the one below, but I prefer the one in below: more simple to understand
 #   [Output("plot1", "children"), Output("plot2", "children")],
  #  [Input("x-dropdown", "value"), Input("luku-dropdown", "value")],
   # [State("plot1", "children"), State("plot2", "children")]
#)

@app.callback([
    Output(component_id="plot1", component_property="children"),
    Output(component_id="plot2", component_property="children")],

    [Input(component_id="x-dropdown", component_property="value"),
     Input(component_id="luku-dropdown", component_property="value")],

    [State("plot1","children"),
    State("plot2","children")
          ])

def saitti_funktio(x, paino, children1, children2):

    #paino = int(paino)

    #print("Callback called with x =", x, "and paino =", paino) # Bugin: If there would be an error
    
    if x == 'CCAFS LC-40' or x == 'VAFB SLC-4E' or x == 'KSC LC-39A' or x == 'CCAFS SLC-40':

        frame = df[df["Launch Site"]==x]

        landing_class_pituus = len(frame["class"])

        onnistuneiden_laskujen_määrä = len(frame[frame["class"]==1])

        epäonnistuneiden_laskujen_määrä = landing_class_pituus - onnistuneiden_laskujen_määrä

        data = {
                "Category": ["Success", "Failure"],
                "Count": [onnistuneiden_laskujen_määrä, epäonnistuneiden_laskujen_määrä]
                }

        df_1 = pd.DataFrame(data)

        fig_pie = px.pie(df_1, names="Category", values="Count", title=f"Launch Results for {x} Site")

        paino_frame = frame[(frame["Payload Mass (kg)"] >= paino[0]) & (frame["Payload Mass (kg)"] <= paino[1])]
        
        fig_scatter = px.scatter(paino_frame, x="Payload Mass (kg)", y="class", color="Booster Version Category")

        fig_scatter.update_layout(
            title=f"Correlation between Payload and Success for Site {x}",
            xaxis_title="Payload Mass (kg)",
            yaxis_title="Class (1 = success, 0 = failure)",
                        )
        
        #print("Returning plots for individual site analysis") # Bugin: If there would be an error
        return [dcc.Graph(figure=fig_pie),
                dcc.Graph(figure=fig_scatter)]

    if x == "All Sites":

        value_counts_1 = df.groupby("Launch Site")["class"].value_counts()

        result_df = value_counts_1.reset_index(name="Count")

        filtered_df = result_df[result_df['class'] == 1]

        fig_pie = px.pie(filtered_df, names="Launch Site", values="Count", title="Total Success Launches By All Sites")

        paino_frame = df[(df["Payload Mass (kg)"] >= paino[0]) & (df["Payload Mass (kg)"] <= paino[1])]

        fig_scatter = px.scatter(paino_frame, x="Payload Mass (kg)", y="class", color="Booster Version Category")

        fig_scatter.update_layout(
            title="Correlation between Payload and Success for All Sites",
            xaxis_title="Payload Mass (kg)",
            yaxis_title="Class (1 = success, 0 = failure)",
                        )
        
        #print("Returning plots for individual site analysis") # Bugin: If there would be an error
        return [dcc.Graph(figure=fig_pie),
                dcc.Graph(figure=fig_scatter)]

if __name__ == '__main__':
    app.run_server()
